<img title="GitHub Octocat" src='./img/Octocat.jpg' style='height: 60px; padding-right: 15px' alt="Octocat" align="left" height="60"> This notebook is part of a GitHub repository: https://github.com/pessini/moby-bikes
<br>MIT Licensed
<br>Author: Leandro Pessini

# <p style="font-size:100%; text-align:left; color:#444444;">Data Wrangling</p>

# <p style="font-size:100%; text-align:left; color:#444444;">Table of Contents:</p>
* [1. Datasets](#1)
  * [1.1 Rentals Data - Moby Bikes](#1.1)
  * [1.2 Weather Data - Met Éireann](#1.2)
* [2. Preprocessing & Feature Engineering](#2)
  * [2.1 Target variable distribution](#2.1)
  * [2.2 Missing values](#2.2)
  * [2.3 Exploratory Analysis](#2.3)
  * [2.4 Features Importance](#2.4)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

<a id="1"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">1- Datasets</p>

Dataset provided by [Moby Bikes](https://data.gov.ie/dataset/moby-bikes) through a public [API](https://data.smartdublin.ie/mobybikes-api). 

Dataset provided by [Met Éireann](https://www.met.ie/) through a public [API](https://data.gov.ie/organization/meteireann).


[Met Éireann Weather Forecast API](https://data.gov.ie/dataset/met-eireann-weather-forecast-api/resource/5d156b15-38b8-4de9-921b-0ffc8704c88e)

<a id="1.1"></a>

## Rentals Data - Moby Bikes

### MongoDB

In [ ]:
# Database
from conn import mongodb
import importlib
from pymongo import MongoClient
from urllib.parse import quote_plus
importlib.reload(mongodb)

In [ ]:
def _connect_mongo(host, port, username, password, db_name):
    """ A util for making a connection to mongo """

    if username and password:
        try:
            mongo_uri = f'mongodb://{username}:{quote_plus(password)}@{host}:{port}/{db_name}'
            conn = MongoClient(mongo_uri)
        except:
            print('Could not connect to MongoDB')
    else:
        conn = MongoClient(host, port)

    return conn

In [ ]:
def read_mongo(query={}, collection='', no_id=True):
    """ Read from Mongo and Store into a DataFrame """
    df = None
    try:
        # Connect to MongoDB
        conn = _connect_mongo(host=mongodb.host, port=mongodb.port, username=mongodb.user_name, password=mongodb.pass_word, db_name=mongodb.db_name)
        db = conn.mobybikes # switch to the database

        if collection in db.list_collection_names():
            
            # Make a query to the specific DB and Collection and store into a Dataframe
            data = db[collection].find(query)
            df =  pd.DataFrame(list(data))
            
            # Delete the _id
            if no_id:
                del df['_id']
        else:
            print(f'Collection {collection} was not found!')
            pass

        # close mongodb connection
        conn.close()
    except:
        print('Could not query MongoDB')
    
    return df


In [ ]:
# historical_data = read_mongo(collection='historical')

### Dataset (csv)

In [ ]:
historical_data = pd.read_csv('../data/raw/combined_csv.csv')

In [ ]:
historical_data.columns = historical_data.columns.str.lower()
historical_data.head()

In [ ]:
historical_data.isnull().sum()

<a id="1.2"></a>

## Weather Data - Met Éireann

About the weather data there are two important decisions. One is about from **which station** the **historical data will be collected** and the other one is about the **frequency of data**, which can be **hourly or daily**.

### Station Name: **PHOENIX PARK**

In [ ]:
phoenixpark_weather_hourly = pd.read_csv('../data/raw/hly175.csv')
phoenixpark_weather_hourly.head()

In [ ]:
phoenixpark_weather_daily = pd.read_csv('../data/raw/dly175.csv')
phoenixpark_weather_daily.head()

### Station Name: **DUBLIN AIRPORT**

In [ ]:
dublin_airport_weather_hourly = pd.read_csv('../data/raw/hly532.csv')
dublin_airport_weather_hourly.head()

### Phoenix Park Station vs Dublin Aiport Station
Geographically, the station at Phoenix Park would be the most suitable choice but unfortunately, they do not collect Wind information which in Ireland plays an important role when deciding to go cycling or not. For those who are not familiar with Irish weather, it rains a lot and mostly we do not have much choice about it but the wind is something that can prevent you go outside or choose a different kind of transport. Heavy rain is kind of rare on the other hand.

### Hourly vs Daily data
On this subject, daily data for the business would make more sense but because the weather is so unpredictable in Ireland (it can completely change in an hour), the best option would be hourly especially if we are looking at a historical perspective. For simplicity and better planning, we can always aggregate the predicted results by day.

In [ ]:
dublin_airport_weather_hourly['date'] = pd.to_datetime(dublin_airport_weather_hourly['date'])
dublin_airport_weather_hourly.info()

In [ ]:
dublin_airport_weather_hourly.tail()

### Sampling

In [ ]:
start_date_hist = datetime(2021, 2, 1) # first day
end_date_hist = datetime(2021, 8, 31) # last day used as historical data

In [ ]:
recent_dubairport_data = dublin_airport_weather_hourly.copy()
recent_dubairport_data = recent_dubairport_data[(recent_dubairport_data.date >= start_date_hist) & (recent_dubairport_data.date <= end_date_hist)]
len(dublin_airport_weather_hourly), len(recent_dubairport_data)

In [ ]:
columns_to_drop = ['ind','ind.1','ind.2','ind.3','vappr','msl','ind.4','wddir','ww','w','sun','vis','clht','clamt']
weather_data = recent_dubairport_data.drop(columns=columns_to_drop)
weather_data.to_csv('../data/interim/hist_weather_data.csv', index=False)

In [ ]:
weather_data.head()

<a id="2"></a>
# <p style="font-size:100%; text-align:left; color:#444444;">2- Preprocessing & Feature Engineering</p>

## Hypothesis

Hourly trend: There must be high demand during office timings. Early morning and late evening can have different trend (cyclist) and low demand during 10:00 pm to 4:00 am.

Daily Trend: Registered users demand more bike on weekdays as compared to weekend or holiday.

Rain: The demand of bikes will be lower on a rainy day as compared to a sunny day. Similarly, higher humidity will cause to lower the demand and vice versa.

Temperature: In Ireland, temperature has positive correlation with bike demand.

Traffic: It can be positively correlated with Bike demand. Higher traffic may force people to use bike as compared to other road transport medium like car, taxi etc.



### New Features
- date (yyyy-mm-dd)
- month
- hour
- workingday
- peak
- holiday
- season
- battery_start
- battery_end
- path? (multi polygon)
- rental_duration


The number of rentals each hour will be aggregate later with a new feature `count`.

In [ ]:
rentals_data = historical_data.drop(['harvesttime','ebikestateid'], axis=1).copy()
rentals_data[["lastgpstime", "lastrentalstart"]] = rentals_data[["lastgpstime", "lastrentalstart"]].apply(pd.to_datetime)

rentals_data = rentals_data.astype({'battery': np.int16}, errors='ignore') # errors ignore to keep missing values (not throwing error)

In [ ]:
rentals_data.info()

### Rentals' information

- `coordinates`: converting latitude and longitude to an array to store a GeoJSON object *MultiPoint* 
- `start_battery`: getting the battery status when the rental started
- `lastgpstime`: new variable that will only store the last record when grouping rentals

In [ ]:
def feat_eng(x):
    d = {}
    d['coordinates'] = x[['latitude','longitude']].values.tolist()
    d['start_battery'] = list(x['battery'])[-1] # get the first battery status (when rental started)
    d['lastgpstime'] = list(x['lastgpstime'])[0] # get the last gpstime (previously sorted)
    
    return pd.Series(d, index=['coordinates', 'start_battery', 'lastgpstime'])

# also sorting data by lastgpstime
grouped_rentals = rentals_data.sort_values("lastgpstime", ascending=False).groupby(['lastrentalstart', 'bikeid']).apply(feat_eng).reset_index()

In [ ]:
grouped_rentals.shape

### Date and time - new features
- `rental_date`
- `rental_month`
- `rental_hour`
- `holiday`
- `workingday`
- `peak`
- `season`: (1 = Spring, 2 = Summer, 3 = Fall, 4 = Winter)
- *`duration`: duration of the rental

\* **Assumption**: Due to lack of information and data, to calculate the average rent time I am assuming that when a new bike rental starts the average will be calculated by: $ ( AvgRentTime* = LastGPSTime - LastRentalStart ) $

In [ ]:
grouped_rentals['rental_date'] = pd.to_datetime(grouped_rentals['lastrentalstart'].dt.date)
grouped_rentals['rental_month'] = grouped_rentals['lastrentalstart'].dt.month
grouped_rentals['rental_hour'] = grouped_rentals['lastrentalstart'].dt.hour

Slicing the dataset to get the sample as per weather data above.

In [ ]:
grouped_rentals = grouped_rentals[(grouped_rentals.rental_date >= start_date_hist) & (grouped_rentals.rental_date <= end_date_hist)]

In [ ]:
# time of rental in minutes (lastgpstime - rental-start)
grouped_rentals['duration'] = (grouped_rentals['lastgpstime'] - grouped_rentals['lastrentalstart']) / pd.Timedelta(minutes=1)

A few GPS records have frozen and stopped sending the accurate data back. About 345 records which would lead to a bias duration of rentals.

To prevent any inaccurate information these records will be set as `NaN`.

In [ ]:
grouped_rentals['duration'] = np.where(grouped_rentals['duration'] < 0, np.NaN, grouped_rentals['duration'])
len(grouped_rentals[ np.isnan(grouped_rentals['duration']) ])

## Bank Holidays

In [ ]:
qry_bh = {
    'type': 'National holiday'
}

bank_holidays = read_mongo(query=qry_bh, collection='irishcalendar')
bank_holidays.drop(['country', 'type'], axis=1, inplace=True)
bank_holidays['date'] = pd.DatetimeIndex(bank_holidays['date'].apply(pd.to_datetime))

In [ ]:
# holiday
grouped_rentals['holiday'] = grouped_rentals['rental_date'].isin(bank_holidays['date'])

# day of the week
grouped_rentals['dayofweek'] = grouped_rentals['rental_date'].dt.dayofweek

# working day (Monday=0, Sunday=6)
grouped_rentals['working_day'] = grouped_rentals['dayofweek'] < 5 # from 0 to 4 or monday to friday

## Seasons

In [ ]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [(3, (datetime(Y,  1,  1),  datetime(Y,  3, 20))),
           (0, (datetime(Y,  3, 21),  datetime(Y,  6, 20))),
           (1, (datetime(Y,  6, 21),  datetime(Y,  9, 22))),
           (2, (datetime(Y,  9, 23),  datetime(Y, 12, 20))),
           (3, (datetime(Y, 12, 21),  datetime(Y, 12, 31)))]

def get_season(date: pd.DatetimeIndex) -> int:
    '''
        Receives a date and returns the corresponded season
        0 - Spring | 1 - Summer | 2 - Autumn | 3 - Winter
        Vernal equinox(about March 21): day and night of equal length, marking the start of spring
        Summer solstice (June 20 or 21): longest day of the year, marking the start of summer
        Autumnal equinox(about September 23): day and night of equal length, marking the start of autumn
        Winter solstice (December 21 or 22): shortest day of the year, marking the start of winter
    '''
    date = date.replace(year=Y)
    return next(season for season, (start, end) in seasons if start <= date <= end)


grouped_rentals['season'] = grouped_rentals.rental_date.map(get_season)

In [ ]:
grouped_rentals.groupby('season').size()

In [ ]:
grouped_rentals.isnull().sum()

## Battery

In [ ]:
grouped_rentals['start_battery'] = pd.to_numeric(grouped_rentals['start_battery'])

In [ ]:
grouped_rentals[grouped_rentals['start_battery'] > 100]

From the battery records there is a few cases that we can consider. Only one record has `> 100` and a few negatives ones. To simplify the analysis the records will be normalized with values between `0 > x > 100`.

All missing values (*n=571*) will not be transformed as it could be only malfunction issue when transmiting the data and it could mislead the analysis.

In [ ]:
# normalize battery status between 0 > x < 100
grouped_rentals['start_battery'] = abs(grouped_rentals['start_battery'])
grouped_rentals.loc[grouped_rentals['start_battery'] > 100, 'start_battery'] = 100

## Peak Times

>https://www.independent.ie/irish-news/the-new-commuter-hour-peak-times-increase-with-record-traffic-volumes-36903431.html

In [ ]:
grouped_rentals['peak'] = grouped_rentals[['rental_hour', 'working_day']] \
    .apply(lambda x: (0, 1)[(x['working_day'] == 1 and (6 <= x['rental_hour'] <= 10 or 15 <= x['rental_hour'] <= 19))], axis = 1)

In [ ]:
new_rentals = grouped_rentals.copy()
new_rentals.to_csv('../data/interim/new_features_rentals.csv', index=False)

## Humidity

Attempt to create a new feature called `Humidity` to avoid Multicollinearity.

### We need different humidity quantities

The problem with relative humidity is that, by itself, it doesn’t really tell you how humid it is.

- **Relative Humidity** – This quantity tells us how close the conditions are to saturation, when condensation of water vapor can occur. The interaction of porous materials with water vapor increases with increasing RH. The chance of growing mold increases with increasing RH, 70% usually given as the threshold to stay below.
- **Dew Point Temperature** – This temperature scales with the amount of water vapor. As more water vapor enters a volume, the dew point goes up. If the air in your crawl space, for example, has a dew point of 75° F, you’re probably going to find condensation somewhere. Look at the water pipes, poorly insulated ducts, and uninsulated duct boots.
- **Wet Bulb Temperature** – If dew point is the temperature of condensation, wet bulb is the temperature of evaporation. Same concept; different direction. This one’s important for cooling our bodies.

Once you get a handle on these three quantities, you’ll have a pretty good understanding of humidity.

> https://www.energyvanguard.com/blog/problem-with-relative-humidity

In [ ]:
weather_data['rhum'] = pd.to_numeric(weather_data['rhum'],errors = 'coerce')
weather_data['wetb'] = pd.to_numeric(weather_data['wetb'],errors = 'coerce')
weather_data['dewpt'] = pd.to_numeric(weather_data['dewpt'],errors = 'coerce')

In [ ]:
weather_data[weather_data.rhum.isnull()]

### Variance Inflation Factor(VIF)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Fill NaN values: 'ffill' -> propagate last valid observation forward to next valid
weather_data.fillna(method="ffill", inplace=True) 

In [ ]:
test_vif = weather_data.copy().drop('date', axis=1)
#Calculate VIF for each variable in the new data frame
vif = pd.DataFrame()
vif["features"] = test_vif.columns
vif["vif_value"] = [variance_inflation_factor(test_vif.values, i) for i in range(test_vif.shape[1])]
vif

To avoid multicolinearity we could create a new feature based on some calculation on those three features related to humidity. Unfortunately, the Forecast API that will be used in production **does not include Wet Bulb Temperature**.

There are a few equations that could applied to calculate Wet Bulb Temperature (eg: Stull formula) but because we do not have which one is used to store this feature on their historical the best would be not use this feature at all.

In [ ]:
#weather_data.drop(['rhum', 'wetb', 'dewpt'], axis=1, inplace=True)
weather_data.drop(['wetb', 'dewpt'], axis=1, inplace=True)

## Combining Rentals and Weather data

In [ ]:
rentals = new_rentals.copy()
weather = weather_data.copy()

weather['rental_date'] = pd.to_datetime(weather['date'].dt.date)
weather['rental_hour'] = weather['date'].dt.hour

In [ ]:
all_data = pd.merge(rentals, weather, on=['rental_date', 'rental_hour'])
all_data.to_csv('../data/interim/all_data.csv', index=False)

## Grouping data to reflect hourly count of rentals

In [ ]:
hourly_rentals = all_data.copy()
count_hourly_rentals = hourly_rentals.groupby(['rental_date', 'rental_hour']).size().reset_index(name='count')
columns_to_drop = ['lastrentalstart','bikeid','coordinates','start_battery','lastgpstime','rental_month','duration','date']
hourly_rentals = hourly_rentals.drop(columns_to_drop, axis=1).drop_duplicates(subset=['rental_date', 'rental_hour'])
hourly_rentals.shape, count_hourly_rentals.shape

In [ ]:
hourly_data = pd.merge(hourly_rentals, count_hourly_rentals, on=['rental_date','rental_hour'])
hourly_data.to_csv('../data/interim/hourly_data.csv', index=False)
hourly_data.head()

<img title="GitHub Mark" src="./img/GitHub-Mark-64px.png" style="height: 32px; padding-right: 15px" alt="GitHub Mark" align="left"> [GitHub repository](https://github.com/pessini/moby-bikes) <br>Author: Leandro Pessini